1. Setup & Environment

In [2]:
import os
from dotenv import load_dotenv

# Strategic RAG Imports (Corrected Paths)
from langchain_mistralai import ChatMistralAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

# These two were moved to 'langchain_classic' in the latest update
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

load_dotenv()
print("✅ Imports successful and Environment loaded.")

✅ Imports successful and Environment loaded.


2. Load the Local Vector Store

In [5]:
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# 1. Get the directory of the notebook, then go one level UP to the root
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

# 2. Build the correct path to your vector store
# This points to: rag-complaint-chatbot/vector_store/faiss_index_sample
vector_store_path = os.path.join(project_root, "vector_store", "faiss_index_sample")

print(f"📁 Looking for vector store at: {vector_store_path}")

# 3. Load the store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

if os.path.exists(vector_store_path):
    vector_store = FAISS.load_local(
        vector_store_path, 
        embeddings, 
        allow_dangerous_deserialization=True
    )
    print("✅ Vector store loaded successfully!")
else:
    print("❌ Still not found! Please check if Task 2 actually created the folder.")

📁 Looking for vector store at: c:\Users\Home-User\rag-complaint-chatbot\vector_store\faiss_index_sample
✅ Vector store loaded successfully!


3. Step-by-Step Retrieval Test

In [7]:
# 1. Initialize the retriever from the loaded vector store
# 'k': 3 tells the engine to find the top 3 most relevant complaint chunks
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("✅ Retriever successfully defined and linked to the Vector Store.")

✅ Retriever successfully defined and linked to the Vector Store.


4. LLM Generation Test

In [8]:
# Initialize the Mistral model
llm = ChatMistralAI(model="open-mistral-nemo", temperature=0.1)

# Define the Senior Strategic Analyst prompt
prompt = ChatPromptTemplate.from_template("""
You are a Senior Strategic Analyst for CrediTrust. 
Answer based ONLY on the provided context. Cite the 'Complaint ID' for every fact.

Context:
{context}

Question: {input}
""")

# Create the chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# Run the test
response = rag_chain.invoke({"input": "Summarize the top billing issues and identify the IDs."})
print("📊 STRATEGIC REPORT:\n")
print(response["answer"])

📊 STRATEGIC REPORT:

Based on the provided context, here are the top billing issues and their corresponding Complaint IDs:

1. **Incorrect Charges:**
   - Complaint ID: Not provided in the context. However, it's mentioned that customers have complained about incorrect charges on their social media channels.

2. **Disputed Transactions:**
   - Complaint ID: Not provided in the context. The customer has mentioned that they have transaction IDs and screenshots of customer support, suggesting they are disputing certain transactions.

3. **Banking Issues:**
   - Complaint ID: Not provided in the context. The customer has mentioned that they have also faced issues with their bank, suggesting potential banking-related billing issues.

4. **Request for Investigation:**
   - Complaint ID: Not provided in the context. The customer has explicitly requested an investigation into their account transactions, indicating a serious billing issue.
